# SELECCIÓN: LISTA DE ELEGIBLES Y DE ESPERA

## Cargar de datos iniciales

In [1]:
import pandas as pd

# Cargar los datos desde el archivo Excel; las hojas 'Cupos' y 'Aspirantes' en DataFrames
file_path = 'INPUT.xlsx'
cupos_programa = pd.read_excel(file_path, sheet_name='Cupos')
cupos_programa["LISTA_ESPERA"] = cupos_programa["CUPOS"].apply(lambda x: x+1 if x<15 else 15)
# cupos_programa["LISTA_ESPERA"] = cupos_programa["CUPOS"]
aspirantes_df = pd.read_excel(file_path, sheet_name='Aspirantes')

## Determinar puntajes diferenciales por prioridad de programas dentro de las opciones
  
   El puntaje diferencial es otorgado a las mujeres que eligieron un programa STEM y es equivalente a 4 puntos adicionales. Solo se aplicará para la obtención de un cupo en la opcion o prioridad donde se haya elegido un programa STEM. 

In [4]:
#Ordenar los aspirantes por puntaje de mayor a menor
aspirantes_df_sorted = aspirantes_df.sort_values(by='Puntaje', ascending=False)

In [6]:
#Definir diccionario con programas STEM
programas_stem = cupos_programa.set_index('PROGRAMAS')['STEM'].to_dict()

#Definir un diccionario para llevar la cuenta de los cupos disponibles por programa
cupos_disponibles_programa = cupos_programa.set_index('PROGRAMAS')['CUPOS'].to_dict()
cupos_disponibles_lista_espera = cupos_programa.set_index('PROGRAMAS')['LISTA_ESPERA'].to_dict()

In [9]:
#Calcular puntajes diferenciales por opción de programa (de 1 a 5)

#Definir función que realice el calculo siempre y cuendo el aspirante sea mujer y el programa en dicha
#opción sea STEM.

def calcular_puntaje_diferencial(row, nombre_opcion):
    # Verifica si el aspirante es mujer y si el programa es STEM
    es_mujer = row['Mujer'] == 1
    programa_es_stem = programas_stem.get(row[nombre_opcion], 0) == 1

    # Si ambas condiciones se cumplen, añade 4 al puntaje
    if es_mujer and programa_es_stem:
        return row['Puntaje'] + 4 #nota: Este 4 debe ser variable
    else:
        return row['Puntaje']

In [10]:
#Aplicar un bucle para que cree 5 columnas adicionales con puntajes diferenciales aplicando la función anterior.
for i in range(1, 6): #Nota este 6 debe ser variable 5+1 por tratarse de range
    opcion = f'Opción {i}'
    aspirantes_df[f'Puntaje Dif {opcion}'] = aspirantes_df.apply(
        lambda row: calcular_puntaje_diferencial(row, opcion), axis=1
    )

## Ordenar de mayor a menor los nuevos puntajes diferenciales en 5 df distintos

In [13]:
resultados_sorted = {}

for i in range(1, 6):
    # Nombres de las columnas relevantes
    opcion_col = f"Opción {i}"
    puntaje_dif_col = f"Puntaje Dif {opcion_col}"

    # Selecciona las columnas relevantes
    puntaje_opcion = aspirantes_df[["ID", "Puntaje", "Mujer", opcion_col, puntaje_dif_col]]

    # Ordena el DataFrame basado en el puntaje diferencial
    puntaje_opcion_sorted = puntaje_opcion.sort_values(by=puntaje_dif_col, ascending=False)

    # Guarda el DataFrame ordenado en un diccionario
    resultados_sorted[opcion_col] = puntaje_opcion_sorted

## Preparar el proceso de seleccion de elegibles y listas de espera

In [14]:
#Inicializar listas para los admitidos y las listas de espera por opción
elegibles_por_opcion = {}
lista_espera_por_opcion = {}

for i in range(1, 6):
    # Creando las listas vacías para cada opción
    elegibles_por_opcion[f'Opción {i}'] = []
    lista_espera_por_opcion[f'Opción {i}'] = []

In [15]:
asignados = set()

def asignar_cupos_elegibles(aspirante, preferencias, elegibles_por_opcion_n, asignados):
    if aspirante['ID'] in asignados:
        return False  # El aspirante ya ha sido asignado, no asignar de nuevo

    for opcion in preferencias:
        if cupos_disponibles_programa[opcion] > 0:
            # Admitir al aspirante en el programa de esta opción
            cupos_disponibles_programa[opcion] -= 1
            elegibles_por_opcion_n.append((aspirante['ID'], opcion))
            asignados.add(aspirante['ID'])  # Registrar al aspirante como asignado
            return True

    return False



def asignar_cupos_lista_espera(aspirante, preferencias, lista_espera_por_opcion_n, asignados):
    if aspirante['ID'] in asignados:
        return False  # El aspirante ya ha sido asignado, no asignar de nuevo

    for opcion in preferencias:
        if cupos_disponibles_lista_espera[opcion] > 0:
            # Admitir al aspirante en el programa de esta opción
            cupos_disponibles_lista_espera[opcion] -= 1
            lista_espera_por_opcion_n.append((aspirante['ID'], opcion))
            asignados.add(aspirante['ID'])  # Registrar al aspirante como asignado
            return True

    return False

## Seleccionar los elegibles

In [16]:
# Procesar cada aspirante para intentar asignarle un cupo en un programa de sus 5 opciones
# Asegurándose de que los diccionarios están inicializados
elegibles_por_opcion = {f'Opción {i}': [] for i in range(1, 6)}
        
for opcion in elegibles_por_opcion:
    for _, aspirante in resultados_sorted[opcion].iterrows():
        preferencias = [aspirante[opcion]]
        asignar_cupos_elegibles(aspirante, preferencias, elegibles_por_opcion[opcion], asignados)

## Seleccionar aspirantes que harán parte de la lista de espera

In [17]:
# Procesar cada aspirante para intentar asignarle un cupo en la lista de espera de alguno de sus 5 opciones
# Asegurándose de que los diccionarios están inicializados
lista_espera_por_opcion = {f'Opción {i}': [] for i in range(1, 6)}
        
for opcion in lista_espera_por_opcion:
    for _, aspirante in resultados_sorted[opcion].iterrows():
        preferencias = [aspirante[opcion]]
        asignar_cupos_lista_espera(aspirante, preferencias, lista_espera_por_opcion[opcion], asignados)

## Consolidar resultados

In [18]:
# A partir de los diccionarios cupos_disponibles_programa, elegibles_por_opcion, lista_espera_por_opcion crear un df

# Crear un diccionario para consolidar la información por programa
datos_por_programa = {programa: [] for programa in cupos_disponibles_programa.keys()}

# Función para agregar aspirantes a datos_por_programa
def agregar_aspirantes(opciones_dict):
    for opcion, aspirantes in opciones_dict.items():
        for aspirante_id, programa in aspirantes:
            datos_por_programa[programa].append(aspirante_id)

# Agregar elegibles y lista de espera a datos_por_programa
agregar_aspirantes(elegibles_por_opcion)
agregar_aspirantes(lista_espera_por_opcion)

# Crear el DataFrame usando el orden de los programas en cupos_disponibles_programa
df_programas = pd.DataFrame(dict([(programa, pd.Series(datos)) for programa, datos in datos_por_programa.items()]))

df_programas


,Programa 1,Programa 2,Programa 3,Programa 4,Programa 5,Programa 6,Programa 7,Programa 8,Programa 9,Programa 10,...,Programa 691,Programa 692,Programa 693,Programa 694,Programa 695,Programa 696,Programa 697,Programa 698,Programa 699,Programa 700
0,Aspirante_51921,Aspirante_32676,Aspirante_14862,Aspirante_49060,Aspirante_58800,Aspirante_56179,Aspirante_58324,Aspirante_19684,Aspirante_34821,Aspirante_26053,...,Aspirante_496,Aspirante_54900,Aspirante_51480,Aspirante_57492,Aspirante_33361,Aspirante_55149,Aspirante_49151,Aspirante_47036,Aspirante_27864,Aspirante_39765
1,Aspirante_36566,Aspirante_38333,Aspirante_44081,Aspirante_58793,Aspirante_19506,Aspirante_44613,Aspirante_59551,Aspirante_45215,Aspirante_58588,Aspirante_3374,...,Aspirante_52250,Aspirante_2926,Aspirante_46943,Aspirante_28627,Aspirante_30143,Aspirante_39197,Aspirante_2830,Aspirante_5357,Aspirante_26131,Aspirante_52676
2,Aspirante_35854,Aspirante_5557,Aspirante_50326,Aspirante_36360,Aspirante_3696,Aspirante_24866,Aspirante_44202,Aspirante_30613,Aspirante_52075,Aspirante_44173,...,Aspirante_52696,Aspirante_41526,Aspirante_59133,Aspirante_26056,Aspirante_29750,Aspirante_50194,Aspirante_53689,Aspirante_46846,Aspirante_26572,Aspirante_20077
3,NaN,Aspirante_56032,Aspirante_6835,Aspirante_5018,Aspirante_58343,Aspirante_54768,Aspirante_25683,Aspirante_20411,Aspirante_49925,Aspirante_37573,...,Aspirante_12503,Aspirante_41293,Aspirante_43272,Aspirante_3426,Aspirante_36155,Aspirante_1869,Aspirante_44739,Aspirante_16005,Aspirante_11642,Aspirante_11532
4,NaN,Aspirante_54675,Aspirante_46687,Aspirante_47029,Aspirante_13619,Aspirante_10685,Aspirante_22113,Aspirante_8751,Aspirante_12529,Aspirante_4905,...,Aspirante_51040,Aspirante_33059,Aspirante_18722,Aspirante_3689,Aspirante_22022,Aspirante_28944,Aspirante_16783,Aspirante_41736,Aspirante_38142,Aspirante_1065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Guardar puntajes diferenciales y resultados en archivos de excel

In [20]:
df_programas.to_excel("OUTPUT_ELEGIBLES_&_ESPERA.xlsx")
aspirantes_df.to_excel("OUTPUT_PUNTAJES_DIFERENCIALES.xlsx")